<a href="https://colab.research.google.com/github/dspraneeth07/AXIOM--SQL-REFLEX-AGENT-v4/blob/main/00_env_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q duckdb pandas numpy networkx faiss-cpu sentence-transformers tqdm
!pip install -q gdown


In [2]:
import duckdb
import pandas as pd
import numpy as np
import networkx as nx

print("Environment ready ✅")


Environment ready ✅


In [3]:
import os

folders = [
    "data",
    "data/spider",
    "data/bird",
    "schemas",
    "agents",
    "outputs"
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("Project folders created ✅")


Project folders created ✅


In [5]:
!mkdir -p data/spider
!wget -q https://raw.githubusercontent.com/taoyds/spider/master/evaluation_examples/examples/dev.json -O data/spider/dev.json
!wget -q https://raw.githubusercontent.com/taoyds/spider/master/evaluation_examples/examples/train_spider.json -O data/spider/train_spider.json
!wget -q https://raw.githubusercontent.com/taoyds/spider/master/evaluation_examples/examples/tables.json -O data/spider/tables.json


In [8]:
!mkdir -p data
!gdown --id 1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J -O spider.zip


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
From (redirected): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J&confirm=t&uuid=4c902584-ffdb-460a-9d08-444141021595
To: /content/spider.zip
100% 206M/206M [00:03<00:00, 64.9MB/s]


In [9]:
!unzip -q spider.zip -d data/spider


In [10]:
import os
os.listdir("data/spider")


['spider_data', '__MACOSX', 'dev.json', 'train_spider.json', 'tables.json']

In [11]:
!git clone https://github.com/AlibabaResearch/DAMO-ConvAI.git data/bird_raw


Cloning into 'data/bird_raw'...
remote: Enumerating objects: 5768, done.
remote: Counting objects: 100% (487/487), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 5768 (delta 309), reused 305 (delta 265), pack-reused 5281 (from 2)
Receiving objects: 100% (5768/5768), 168.81 MiB | 16.06 MiB/s, done.
Resolving deltas: 100% (1916/1916), done.
Updating files: 100% (4894/4894), done.


In [12]:
os.listdir("data/bird_raw")


['SDPO',
 'unid2t',
 'FlowBench',
 'space-3',
 'EPO',
 's2sql',
 'sunsql',
 'proton',
 'cgodial',
 'LICENSE',
 'star',
 'api-bank',
 'dstc11-simmc',
 'bird',
 '.gitignore',
 'dial2vec',
 'dialogue-cse',
 'mmevol',
 'r2sql',
 'doc2bot',
 'metaretriever',
 'diana',
 'tkk',
 'OpenOmni',
 'dater',
 'dial-start',
 'oltqa',
 'acl23doc2dial',
 'WideDeep',
 'spokenwoz',
 '.github',
 'PRO',
 'rlp',
 'OmniCharacter',
 'pcll',
 'MaskedThought',
 'space-1',
 'README.md',
 'ssll',
 'Loong',
 '.git',
 'IOPO',
 'space-2',
 '.DS_Store',
 'sdsql',
 'deep-thinking',
 'graphix',
 'cats',
 'pace',
 'Awesome-Self-Evolution-of-LLM',
 'attention-buckets',
 'spectra']

In [13]:
import duckdb

db = duckdb.connect("axiom_v4.duckdb")
print("DuckDB initialized ✅")


DuckDB initialized ✅


In [17]:
import sqlite3

sqlite_db = "data/spider/spider_data/database/academic/academic.sqlite"
sqlite_conn = sqlite3.connect(sqlite_db)

tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table'",
    sqlite_conn
)
tables


,name
0,author
1,conference
2,domain
3,domain_author
4,domain_conference
5,journal
6,domain_journal
7,keyword
8,domain_keyword
9,publication


In [18]:
table_name = tables.iloc[0]["name"]
df = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 100", sqlite_conn)

db.execute(f"CREATE OR REPLACE TABLE academic_{table_name} AS SELECT * FROM df")
print("Spider DB loaded into DuckDB ✅")


Spider DB loaded into DuckDB ✅


In [19]:
db.execute("SHOW TABLES").fetchdf()


,name
0,academic_author


In [20]:
db.execute("SELECT COUNT(*) FROM academic_" + table_name).fetchone()


(0,)

DATA CONTRACTS (v4)

SchemaGraph:
{
  "tables": [
    {
      "table_name": str,
      "columns": [str],
      "primary_key": [str],
      "foreign_keys": [
        {
          "column": str,
          "ref_table": str,
          "ref_column": str
        }
      ]
    }
  ]
}

AgentInput:
{
  "question": str,
  "candidate_tables": [str],
  "schema_graph": object
}

AgentOutput:
{
  "sql": str,
  "confidence": float,
  "reason": str
}
